In [ ]:
# %pip install -r requirements.txt # type: ignore

In [1]:
from datasets import load_dataset
import pandas as pd
import os
import re
import string
from tqdm import tqdm

In [2]:
from nltk.tokenize import word_tokenize
import nltk


In [3]:
nltk.data.path.append(os.path.abspath('nltk_data'))

In [4]:

# nltk.download('stopwords' , download_dir='nltk_data')
# nltk.download('wordnet' , download_dir='nltk_data')

---


# DATASET


In [5]:
main_path = os.path.abspath('../../Datasets/ANOTASI/HasilAnotasiPutusan.csv')

In [6]:
main_df = pd.read_csv(main_path)

In [7]:
main_df

,doc,fragment,token,tag
0,doc:1,fragment:1,PUTUSAN,O
1,doc:1,fragment:1,Nomor,O
2,doc:1,fragment:1,161,B_VERN
3,doc:1,fragment:1,/,I_VERN
4,doc:1,fragment:1,Pid,I_VERN
...,...,...,...,...
2537884,doc:345,fragment:50030,",",O
2537885,doc:345,fragment:50030,S,O
2537886,doc:345,fragment:50030,.,O
2537887,doc:345,fragment:50030,H,O


In [8]:
print(main_df[0:1])

     doc    fragment    token tag
0  doc:1  fragment:1  PUTUSAN   O


In [9]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2537889 entries, 0 to 2537888
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   doc       object
 1   fragment  object
 2   token     object
 3   tag       object
dtypes: object(4)
memory usage: 77.5+ MB


In [10]:
main_df[0:6]

,doc,fragment,token,tag
0,doc:1,fragment:1,PUTUSAN,O
1,doc:1,fragment:1,Nomor,O
2,doc:1,fragment:1,161,B_VERN
3,doc:1,fragment:1,/,I_VERN
4,doc:1,fragment:1,Pid,I_VERN
5,doc:1,fragment:1,.,I_VERN


---
